## <CENTER> ЧАСТЬ 1
## <CENTER> ПРЕДОБРАБОТКА ДАННЫХ

В нашем распоряжении есть данные о доходах, заболеваемости, социально незащищённых слоях населения России и другие экономические и демографические данные. Познакомимся с ними поближе

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.filterwarnings('ignore') 

Все таблицы будем обрабатывать по одному сценарию:

- переименовывать коллонки, если необходимо;
- проверять на правильность написания регионов;
- проверять на наличие пропусков;
- смотреть на максимальные и минимальные значения;
- визуализировать данные;
- делать выводы.

Чтобы в итоге иметь одну большую объединенную таблицу, нам необходимо чтобы в каждой отдельной все регионы были одинаково написаны, Не всегда один и тот же регион пишется одинаково в разных датасетах. 

Такое очень часто происходит с написаниями регионов РФ в разных документах, а также в других странах (их нет в данной задаче). Это необходимо учитывать при объединении датасетов. Особенностью Тюменской области является то, что в её состав входят два других субъекта Российской Федерации: Ханты-Мансийский автономный округ — Югра и Ямало-Ненецкий автономный округ. 

При этом чаще всего в датасетах указывается отдельно Тюменская область и отдельно — без учёта автономных округов. То же самое касается Архангельской области — в её состав входит самостоятельный субъект Ненецкий автономный округ. Способы написания такие же, как с Тюменской областью.

На сегодняшний день в РФ существует 89 регионов, но тк большинство данных у нас до 2021 года, то будем ориентироваться на 85 регионов. Чтобы все регионы были в едином формате, я сделала специальную маску для проверки.

Возьмем данные с 2000 до 2021 года. Все остальное удалим, а недостающие данные будем заполнять медианой за ближайший к пропускам периоду. Далее возьмем среднее значение за эти года. Это и будет таблица для дальнейшей работы.

## Данные:

- `child_mortality_rural_1990_2021.xls`—число умерших на первом году жизни детей за год, по всем регионам, в сельской местности.
- `child_mortality_urban_1990_2021.xls`—числоумерших на первом году жизни детей за год, по всем регионам, в городской местности.
- `disabled_total_by_age_2017_2022.csv` — число людей с инвалидностью по регионам, по месяцам, по возрастным группам.
- `morbidity_2005_2020_age_disease.xls` — заболеваемость на 100 тыс. человек населения, по возрастным группам и группам заболеваний.
- `poverty_percent_by_regions_1992_2020.csv` — процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам.
- `welfare_expense_share_2015_2020`—расходы на социальную политику от общих расходов бюджета региона, % в год*.
- `cash_real_income_wages_2015_2020` — среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная зарплата, по регионам*.
- `poverty_socdem_20*.xls` — распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам.
- `housing_2020` — характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого помещения, обследование 2020 года*.
- `population.xlsx` — численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг.
- `gross_regional_product_1996_2020.xls`—валовойрегиональныйпродукт на душу населения, в рублях.
- `regional_production_*_*.csv` — объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности за 2005–2016 гг., 2017–2020 гг. (в тысячах рублей, значение показателя за год, полный круг).
- `retail_turnover_per_capita_2000_2021.xls` — оборот розничной торговли на душу населения, в рублях.
- `drug_alco` — сведения о заболеваемости алкоголизмом и наркоманией, на 100 тыс. населения (2005–2018).
- `newborn_2006_2022_monthly.csv` — рождённые в этом месяце, по регионам, без учёта мертворождённых.
- `Папка crimes` — сведения о преступлениях, совершённых отдельными категориями лиц за 2016–2022 гг., по месяцам, регионам, категориям лиц, категориям преступлений.


### `1. Число умерших на первом году жизни детей за год, по всем регионам, в сельской местности`


In [108]:
#Познакомимся с данными:
child_rural = pd.read_excel('data/child_mortality_rural_1990_2021.xls', skiprows=2)
child_rural.head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Российская Федерация,Оба пола,сельское население,11186.0,10943.0,10111.0,9840.0,9010.0,8582.0,7983.0,...,5463.0,5059.0,4681.0,4165.0,3568.0,3032.0,2521.0,2122.0,1916.0,1852.0
1,Центральный федеральный округ,Оба пола,сельское население,1615.0,1550.0,1361.0,1426.0,1347.0,1321.0,1200.0,...,625.0,608.0,487.0,490.0,426.0,347.0,291.0,211.0,228.0,212.0
2,Белгородская область,Оба пола,сельское население,103.0,92.0,75.0,79.0,80.0,72.0,72.0,...,43.0,48.0,41.0,42.0,36.0,34.0,33.0,16.0,22.0,20.0


После очистки и преобразований наша таблица будет выглядеть так:

In [109]:
df_child_mortality_rural = pd.read_csv('data/total/01_df_child_mortality_rural.csv')
df_child_mortality_rural.head(3)

,Регион,Child_mortality_rural
0,Белгородская область,39.59
1,Брянская область,42.09
2,Владимирская область,27.95


**Вывод:
Самое высокое число умерших на первом году жизни детей за год, по всем регионам, в сельской местности принадлежит `Республике Дагестан` (352, а это в 7 раз больше среднего значения), самое низкое- `Магаданской области`(1, а это в 300раз меньше чем в Дагестане). Можно заметить общую тенденцию к снижению по всем регионам из года в год. Что скорее всего можно обяснить улучшением качества жизни, появлением новых лекарств и тд.
Среднее значение по всем регионам: 53**


### `2.Числоумерших на первом году жизни детей за год, по всем регионам, в городской местности.`

In [110]:
child_urban = pd.read_excel('data/child_mortality_urban_1990_2021.xls', skiprows=2)
display(child_urban.head(3))

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Российская Федерация,Оба пола,городское население,23902.0,21549.0,19097.0,18106.0,17131.0,16258.0,14842.0,...,10843.0,10418.0,9641.0,8499.0,7860.0,6545.0,5723.0,5206.0,4573.0,4664.0
1,Центральный федеральный округ,Оба пола,городское население,5317.0,4734.0,4175.0,3899.0,3871.0,3631.0,3254.0,...,2789.0,2735.0,2402.0,2228.0,2146.0,1785.0,1593.0,1409.0,1153.0,1192.0
2,Белгородская область,Оба пола,городское население,209.0,198.0,165.0,165.0,153.0,131.0,102.0,...,84.0,68.0,62.0,68.0,72.0,43.0,40.0,23.0,25.0,34.0


После очистки и преобразований наша таблица будет выглядеть так:

In [111]:
df_child_mortality_urban = pd.read_csv('data/total/02_df_child_mortality_urban.csv')
df_child_mortality_urban.head(3)

,Регион,Child_mortality_urban
0,Белгородская область,63.64
1,Брянская область,70.18
2,Владимирская область,84.77


**Вывод: А вот теперь первое место у Москвы (783) и Московской области, скорее всего это связано с количеством живущих людей в Москве, которое в разы превосодит другие города. И количество в Москве в 7 раз выше среднего значения по всем регионам. Самое низкое колличество у Ненецкого автономного округа.**

### `3. Число людей с инвалидностью по регионам, по месяцам, по возрастным группам.`

In [112]:
disabled = pd.read_csv('data/disabled_total_by_age_2017_2022.csv')
display(disabled.head(3))

,region,total,18_30,31_40,41_50,51_60,60_,date
0,Российская Федерация,11640873.0,550895.0,766054.0,1030652.0,2135436.0,7157836.0,2017-01-01
1,Центральный федеральный округ,3420310.0,118579.0,172662.0,257484.0,598102.0,2273483.0,2017-01-01
2,Белгородская область,223030.0,6318.0,10383.0,16596.0,37444.0,152289.0,2017-01-01


После преобразований и разделении на 4 таблицы по возрастам, мы получили следующие таблицы:

In [113]:
df_disabled_total = pd.read_csv('data/total/03_df_disabled_total.csv')
df_disabled_total.head(3)

,Регион,Disabled_total
0,Алтайский край,1375675.41
1,Амурская область,1086782.00
2,Архангельская область,1142382.68


**Вывод: Самое большое число людей с инвалидностью приходится на Москву(3670565), меньше всего на Чукотку(913917)**

In [114]:
df_disabled_18_30 = pd.read_csv('data/total/03_df_disabled_18_30.csv')
df_disabled_18_30.head(3)

,Регион,Disabled_18_30
0,Алтайский край,58804.00
1,Амурская область,47368.82
2,Архангельская область,46655.18


**Вывод: Самое большое число молодых людей от 18 до 30 лет с инвалидностью приходится на республику Дагестан(137880), меньше всего на Ненецкий автономный округ(54428)**

In [115]:
df_disabled_31_60 = pd.read_csv('data/total/03_df_disabled_31_60.csv')
df_disabled_31_60.head(3)

,Регион,Disabled_31_60
0,Алтайский край,434568.73
1,Амурская область,348660.59
2,Архангельская область,354468.23


**Вывод: Самое большое число людей от 31 до 60 лет (работающий возраст)с инвалидностью приходится на Москву(975683), меньше всего на Чукотку(913917)**

In [116]:
df_disabled_61_ = pd.read_csv('data/total/03_df_disabled_61_.csv')
df_disabled_61_.head(3)

,Регион,Disabled_61_
0,Алтайский край,849579.23
1,Амурская область,658029.14
2,Архангельская область,708535.82


**Вывод: Самое большое число людей с инвалидностью старше 61 года (пенсионеры) приходится на Москву(2542835), меньше всего на Чукотку(551904)**

### `4. Заболеваемость на 100 тыс.чел. населения, по возрастным группам и группам заболеваний`

In [117]:
morbidity = pd.read_excel('data/morbidity_2005_2020_age_disease.xls', skiprows=2)
display(morbidity.head(3))

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Российская Федерация,"Беременность, роды и послеродовой период",0-14 лет,21.3,17.6,17.9,22.1,18.1,18.9,16.2,17.7,12.2,12.1,8.1,15.5
1,Российская Федерация,"Беременность, роды и послеродовой период",15-17 лет,1537.3,1527.5,1605.2,1681.5,1618.9,1451.3,1247.6,1229.1,1272.3,1186.5,914.0,794.0
2,Российская Федерация,"Беременность, роды и послеродовой период",18 лет и старше,6731.7,6833.5,7229.7,7516.9,8007.5,8084.4,7981.6,8141.8,8084.6,8219.9,7734.1,7278.6


После преобразования я сделала три таблицы по видам заболеваний:
- общее количество,
- злокачественные образования,
- Психические расстройства и расстройства поведения

In [118]:
df_morbidity_total = pd.read_csv('data/total/04_df_morbidity_total.csv')
df_morbidity_total.head(3)

,Регион,Morbidity_total
0,Алтайский край,56336.62
1,Амурская область,41204.68
2,Архангельская область,19127.44


**Вывод: Самое большое количество среди всех заболеваний суммарно наблюдается в Ненецком автономном округе (в 2 раза больше среднего значения), меньше всего в Севастополе (в 4 раза меньше среднего), а также в Адыгее, Крыму и Тюменской области.**

In [119]:
df_morbidity_onko = pd.read_csv('data/total/04_df_morbidity_onko.csv')
df_morbidity_onko.head(3)

,Регион,Morbidity_onko
0,Алтайский край,2592.75
1,Амурская область,2584.06
2,Архангельская область,2585.93


**Вывод: Самое большое количество Злокачественных новообразований наблюдается в Воронежской области, меньше всего в Чеченской республике. Но в целом можно сказать что все значения примерно одинаковые, т.е. близкие к среднему**

In [120]:
df_morbidity_psiho = pd.read_csv('data/total/04_df_morbidity_psiho.csv')
df_morbidity_psiho.head(3)

,Регион,Morbidity_psiho
0,Алтайский край,2107.83
1,Амурская область,1584.90
2,Архангельская область,2317.27


**Вывод: Самое большое количество психических расстройств и расстройств поведения наблюдается в Астраханской области (в 2 раза больше среднего значения), меньше всего в Республике Тыва.**

### `5.Процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума),оценка за год по регионам.`

In [121]:
poverty = pd.read_csv('data/poverty_percent_by_regions_1992_2020.csv')
display(poverty.head(3))

,region,year,poverty_percent
0,Российская Федерация,1992,33.5
1,Российская Федерация,1993,31.3
2,Российская Федерация,1994,22.4


После преобразований наша таблица выглядит так:

In [122]:
df_poverty = pd.read_csv('data/total/05_df_poverty.csv')
df_poverty.head(3)

,Регион,Poverty_percent
0,Алтайский край,24.25
1,Амурская область,24.29
2,Архангельская область,18.82


**Вывод: Процент людей, живущих за чертой бедности выше всего в Ингушетии (41%), а также Калмыкия и Тыва, а ниже всего в Ямало-Ненецком автономном округе(7%). Среднее значение 19%**

### `6. Расходы на социальную политику от общих расходов бюджета региона, % в год`

In [123]:
welfare = pd.read_excel('data/welfare_expense_share_2015_2020.xlsx')
display(welfare.head())

,region,2015,2016,2017,2018,2019,2020
0,Российская Федерация,15.8,16.6,20.4,20.3,19.8,21.3
1,Центральный\n федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN
2,Белгородская область,11.3,11.8,16.4,15.6,13.8,16.1
3,Брянская область,22.0,26.6,24.7,23.7,20.7,23.1
4,Владимирская область,18.1,18.0,24.0,23.4,23.4,26.0


После преобразований наша таблица выглядит так:

In [124]:
df_welfare = pd.read_csv('data/total/06_df_welfare.csv')
df_welfare.head(3)

,Регион,Welfare
0,Белгородская область,15.98
1,Брянская область,18.51
2,Владимирская область,18.15


**Вывод: Самый большой % расходов на социальную политику от общего числа приходится на Чеченскую республику(20%), меньше всего в Чукотском автономном округе(14%), среднее значение 18%**

### `7. Cреднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная зарплата, по регионам`

In [125]:
cash = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx')
display(cash.head(3))

,region,2015,2016,2017,2018,2019,2020
0,Российская Федерация,302541.0,30865.0,31897.0,33266.0,35338.0,36073.0
1,Центральный\n федеральный округ,38832.0,40200.0,41897.0,44009.0,47252.0,48226.0
2,Белгородская область,28043.0,29799.0,30342.0,30778.0,32352.0,32841.0


После преобразований наша таблица выглядит так:

In [126]:
df_cash = pd.read_csv('data/total/07_df_2cash.csv')
df_cash.head(3)

,Регион,Cash_income
0,Белгородская область,30692.50
1,Брянская область,26015.50
2,Владимирская область,23908.33


**Вывод:
Средние доходы населения c 2015 года до 2020 года выросли на 16%, в то время как максимальные изменились почти на 20%.
Самые высокие доходы принадлежат Ямало_Ненецкому автономному округу (39631), за ним идет Чукотка, Ненецкий автономный округ и Москва, а самые маленькие-  Республике Ингушетия ( 22720). Средние ежегодные доходы за все 6 находятся в пределах 30тр. 64 региона из 87 имеют доход ниже среднего уровня ( а это 74%)**

### `8. Распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам.`

У нас 4 разные таблицы по годам:

In [127]:
socdem_2017 = pd.read_excel('data/poverty_socdem_2017.xls', skiprows=2)
socdem_2018 = pd.read_excel('data/poverty_socdem_2018.xls', skiprows=2)
socdem_2019 = pd.read_excel('data/poverty_socdem_2019.xls', skiprows=2)
socdem_2020 = pd.read_excel('data/poverty_socdem_2020.xls', skiprows=2)
display(socdem_2017.head(3))

,Unnamed: 0,Все население,Дети в возрасте до 16 лет,Население старше трудоспособного возраста,Население трудоспособного возраста
0,Российская Федерация,100,39.3,6.6,54.1
1,Белгородская область,100,43.4,11.8,44.8
2,Брянская область,100,42.9,4.9,52.2


После всех преобразований я сделала 3 таблицы по разным группам со средними значениями в период 2017-2020 гг. Как все другие таблицы делать эту, дополняя недостающие года я не стала, т.к. в самом конце получались очень далекие от дейтсвительности данные, слишком усредненные и не дающие никакой информативности.

In [128]:
df_socdem_deti = pd.read_csv('data/total/08_df_socdem_deti_2.csv')
df_socdem_deti.head(3)

,Регион,Socdem_deti
0,Белгородская область,31.87
1,Брянская область,41.38
2,Владимирская область,39.45


**Вывод: Самый большой % малоимущего насления группы ДЕТИ приходится на Ямало-Ненецкий автономный округ(40%), и это странно, ведь  там достаточно большие доходы благодаря нефтенным компаниями и тд. Видимо для коренных жителей, которые живут никак не связанно с этой промышленностью достаточно большая часть. Меньше всего в Ленинградской области(35%).**

In [129]:
df_socdem_trud = pd.read_csv('data/total/08_df_socdem_trud_2.csv')
df_socdem_trud.head(3)

,Регион,Socdem_trud
0,Белгородская область,35.47
1,Брянская область,54.78
2,Владимирская область,53.65


**Вывод: Самый большой % малоимущего населения группы Трудящихся приходится на Чувашскую республику(57%), а самый маленький на Белгородскую область (51%) и Чукотку(видимо там все хорошо зарабатывают из-за нефтедобывающей промышленности. Cреднее значение 55%**

In [130]:
df_socdem_pens = pd.read_csv('data/total/08_df_socdem_pens_2.csv')
df_socdem_pens.head(3)

,Регион,Socdem_pens
0,Белгородская область,7.35
1,Брянская область,3.88
2,Владимирская область,6.60


**Вывод: Самый большой % малоимущего населения категории Пенсионеры приходится на Кабардино-Балкарию (7%), Смоленскую область и как ни странно город Москва, а самый маленький % в Ямало-ненецком автономном округе(5%).**

### `9. Характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого помещения, обследование 2020 года`

In [131]:
housing = pd.read_excel('data/housing_2020.xlsx')
display(housing.head(3))

,Регион,"Число домохозяйств, проживающих во всех типах жилых помещений","в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности","в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность","в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность",затруднились ответить,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство
0,Российская Федерация,100.0,80.7,15.4,3.8,0.1,24.2,16.9,2.4
1,Центральный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Белгородская область,100.0,86.3,11.7,2.0,0.0,25.1,17.8,2.6


После преобразований я оставила только 1 признак по жилой площади. Дополнять данные другими годами также не было смысла.

In [132]:
df_housing_live = pd.read_csv('data/total/09_df_housing_live.csv')
df_housing_live.head(3)

,Регион,Housing_live
0,Белгородская область,17.8
1,Брянская область,24.1
2,Владимирская область,16.1


**Выводы: Мы располагем только данныеми за 2020 год, но предположим что это и будет наше среднее значение. самый большой размер жилой лощади (почти 28) приходится на Республику Ингушетия, меньше всего Республика Тыва.(13,6), Москва находится рядом (13,8)**

### `10. Численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг.`

In [133]:
population = pd.read_excel('data/population.xlsx', skiprows=1)
display(population.head(3))

,Unnamed: 0,Unnamed: 1,январь 1999 г.,январь 2000 г.,январь 2001 г.,январь 2002 г.,январь 2003 г.,январь 2004 г.,январь 2005 г.,январь 2006 г.,...,январь 2013 г.,январь 2014 г.,январь 2015 г.,январь 2016 г.,январь 2017 г.,январь 2018 г.,январь 2019 г.,январь 2020 г.,январь 2021 г.,январь 2022 г.
0,все население,w2:p_mest:11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Российская Федерация,643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Раздел 1. Муниципальные образования субъектов ...,00000000000,147539426.0,146890128.0,146303611.0,145649334.0,144963650.0,144333586.0,143801046.0,143236582.0,...,143347059.0,143666931.0,146267288.0,146544710.0,146804372.0,146880432.0,146780720.0,146748590.0,146171015.0,145557576.0


После всех преобразований таблица выглядит так (среднее количество в период с 2000 по 2021гг:

In [134]:
df_population = pd.read_csv('data/total/10_df_population.csv')
df_population.head(3)

,Регион,Population
0,Белгородская область,1530937.32
1,Брянская область,1276076.05
2,Владимирская область,1437729.27


**Вывод: Лидером конечно же является Москва(11619036), несмотря на то что среднее значение за 20 лет сильно отличается от сегодняшней ситуации. Меньше всего в Ненецком автономном округе(42696). Среднее значение 1720844**

### ` 11. Валовой региональный продукт на душу населения, в рублях.`

In [135]:
grp = pd.read_excel('data/gross_regional_product_1996_2020.xls', skiprows=2)
display(grp.head(3))

,Unnamed: 0,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Российская Федерация,12225.0,13938.4,15371.1,26200.6,39532.3,49474.8,60611.4,74840.5,97691.9,...,317515.3,348641.5,377006.0,405147.7,449097.9,505337.7,543065.9,614333.2,646144.1,640519.0
1,Центральный федеральный округ,NaN,NaN,16564.4,31118.7,48205.0,58851.5,75739.2,94244.6,121487.7,...,417288.1,451517.2,494482.7,536607.9,580706.6,663881.8,711031.6,792780.9,835858.8,854978.5
2,Белгородская область,9575.6,10792.2,12242.8,21398.0,27969.5,33126.7,41327.4,50271.4,75629.4,...,331010.0,354570.6,368874.8,400820.8,447619.7,501467.8,539720.5,588641.5,617426.5,646569.0


После всех преобразований наша таблица выглядит так:

In [136]:
df_grp = pd.read_csv('data/total/11_df_grp.csv')
df_grp.head(3)

,Регион,GRP
0,Белгородская область,296532.88
1,Брянская область,159780.83
2,Владимирская область,188789.51


**Вывод: За годы Валовой Региональный продукт вырос в 60 раз. Самый большой  в Ненецком автономном округе(2439921), рядом Ямало-ненецкий округ, Сахалин и Чукотка , а самый маленький в Крыму(83488). Главные причины такого превышения заключаются в наличии на территории этих регионов крупнейшего производства нефтегазовых ресурсов, обеспечивающего высокие доходы. Известно, что валовой региональный продукт (валовая добавленная стоимость) включает прибыль организаций, заработную плату наемных работников, предпринимательские доходы, поступления в амортизационные фонды организаций, доходы государства в виде налогов. Поэтому Москва и Санкт-Петербург также входит в ТОП 10**

### `12. Объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности за 2005–2016 гг., 2017–2020 гг. (в тысячах рублей, значение показателя за год, полный круг).`

In [137]:
reg_prod__2005_2016 = pd.read_csv('data/regional_production_2005_2016.csv')
display(reg_prod__2005_2016.head(3))

,region,production_field,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Российская Федерация,РАЗДЕЛ С ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,3.062460e+09,3.720887e+09,4.488915e+09,5.271733e+09,5.090973e+09,6.217952e+09,8.020217e+09,8.950066e+09,9.213745e+09,9.690978e+09,1.125954e+10,1.173050e+10
1,Российская Федерация,Подраздел СА ДОБЫЧА ТОПЛИВНО-ЭНЕРГЕТИЧЕСКИ...,2.686256e+09,3.292691e+09,3.983633e+09,4.650669e+09,4.537654e+09,5.470740e+09,7.032308e+09,7.921745e+09,8.160342e+09,8.563862e+09,9.896535e+09,1.016955e+10
2,Российская Федерация,"Добыча каменного угля,бурого угля и торфа",2.321794e+08,2.264506e+08,2.862991e+08,4.414241e+08,3.485582e+08,5.089025e+08,6.663161e+08,6.514620e+08,6.003871e+08,6.222425e+08,7.721283e+08,9.090367e+08


In [138]:
reg_prod__2017_2020 = pd.read_csv('data/regional_production_2017_2020.csv')
display(reg_prod__2017_2020.head(3))

,region,production_field,2017,2018,2019,2020
0,Российская Федерация,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,1.391616e+10,1.819387e+10,1.832413e+10,1.449890e+10
1,Российская Федерация,Добыча угля,1.228407e+09,1.567151e+09,1.392654e+09,1.100662e+09
2,Российская Федерация,Добыча и обогащение угля и антрацита,1.151523e+09,1.472893e+09,1.302804e+09,1.012754e+09


После всех преобразований я оставила 3 вида производства: 'ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ', 'ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА', 'ВСЕГО':

In [139]:
#Преобразованная таблица:
df_reg_prod_mining = pd.read_csv('data/total/12_df_reg_prod_mining.csv')
display(df_reg_prod_mining.head(3))

,Регион,Reg_prod_mining
0,Алтайский край,5418837.75
1,Амурская область,58775378.13
2,Архангельская область,29868057.62


In [140]:
df_reg_prod_manufactor = pd.read_csv('data/total/12_df_reg_prod_manufactor.csv')
df_reg_prod_manufactor.head(3)

,Регион,Reg_prod_manufactor
0,Алтайский край,2.543652e+08
1,Амурская область,2.741230e+07
2,Архангельская область,2.713518e+08


In [141]:
df_reg_prod_total = pd.read_csv('data/total/12_df_reg_prod_total.csv')
df_reg_prod_total.head(3)

,Регион,Reg_prod_total
0,Алтайский край,5.031198e+08
1,Амурская область,2.002126e+08
2,Архангельская область,5.093577e+08


**Выводы: Москва является неоспоримым лидером по суммарному объёму отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, по видам деятельности.Потому что именно на Москву приходится больше всего Обрабатывающих производств, это и повлияло на суммарный рейтиинг. Меньше всего данный показатель в Республике Ингушетия. Как и следовало ожидать лидером по Добыче полезных ископаемых является Ханты-Мансийский автономный округ - Югра, а меньше всего в Мордовии.**

### `13. Оборот розничной торговли на душу населения, в рублях.`

In [142]:
retail = pd.read_excel('data/retail_turnover_per_capita_2000_2021.xls', skiprows=2)
display(retail.head(3))

,Unnamed: 0,Unnamed: 1,2000,2001,2002,2003,2004,2005,2006,2007,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,NaN,NaN,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,...,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год,значение показателя за год
1,Российская Федерация,рубль,16046,21031,25913,31557,39166,49063,60901,76111,...,149401,165051,180410,188017,192541,202568,215074,229104,231283,270606
2,Центральный федеральный округ,рубль,26062,33630,40010,46760,57011,68302,81774,97965,...,188757,207394,231873,237737,243426,258278,277011,296368,301192,354141


После всех преобразований наша таблица выглядит так:

In [143]:
#Преобразованная таблица:
df_retail = pd.read_csv('data/total/13_df_retail.csv')
display(df_retail.head(3))

,Регион,Retail
0,Белгородская область,114915.23
1,Брянская область,109399.95
2,Владимирская область,87961.45


**Вывод: Лидером конечно же является Москва, несмотря на то что среднее значение за 20 лет сильно отличается от сегодняшней ситуации, на втором месте Сахалин, тк там очень дорогая доставка. Меньше всего в Республика Ингушетия автономном округе**

### `14. Сведения о заболеваемости алкоголизмом и наркоманией, на 100 тыс. населения (2005–2018):`

➔ Речь идёт о впервые установленном диагнозе, в данном случае — о постановке на учёт в диспансере. То есть если в прошлом году человек был признан алкоголиком, в этом году он уже не учитывается в статистике, если стоит на учёте или вылечился и был снят с учёта. Также если человек не попал на учёт, хотя соответствует критериям постановки соответствующего диагноза, он не будет учтён в данной статистике. Таким образом, эти цифры занижены. Кроме того, так как показатель приводится именно по впервые выявленным случаям, в теории его можно аккумулировать (но с оговорками).
 
➔ Показатель на 100 тысяч населения — привычный показатель в эпидемиологии. Однако очевидно, что в данном случае он рассчитывается по всей популяции, хотя, возможно, такой диагноз ставится чаще в некоторых когортах.

➔ В файле четыре листа. Отдельно приводятся показатели за 2017–2018 гг., так как взяты из другого источника, и там немного иначе написаны названия регионов.

Откроем файлы по заболеваемости алкоголизмом:

In [144]:
alco_2005_2016 = pd.read_excel('data/drug_alco.xlsx', sheet_name='alco')
alco_2017_2018 = pd.read_excel('data/drug_alco.xlsx', sheet_name='alco1718')
alco_2005_2016.head(3)

,region,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Российская Федерация,147.4,135.3,123.1,122.2,112.5,108.5,96.7,85.8,78.3,74.6,70.9,64.9
1,Центральный федеральный округ,141.5,131.1,120.3,119.8,109.6,103.3,86.4,80.0,72.9,69.1,64.1,55.3
2,Белгородская область,99.6,93.2,84.0,92.3,94.3,86.4,72.6,74.8,66.9,56.8,50.7,46.1


После всех преобразований и обьединений наша таблица выглядит так:

In [145]:
df_alco = pd.read_csv('data/total/14_df_alco.csv')
df_alco.head(3)

,Регион,Alco
0,Белгородская область,91.07
1,Брянская область,145.76
2,Владимирская область,117.60


**Вывод: Лидером по заболеваемости алкоголизмом является Чукотский автономный округ(331, а это в 3 раза выше среднего значения), дальше идут Магадан, Сахалин и Ненецкий автономный округ. Меньше всего в Руспублике Ингушетия. За эти годы заболиеваемость алкоголизмом в России снизилась почти в 3 раза.**

Проделаем те же дейтсвия с данными по заьолеванию наркоманией.

In [146]:
drug_2005_2016 = pd.read_excel('data/drug_alco.xlsx', sheet_name='drugs')
drug_2017_2018 = pd.read_excel('data/drug_alco.xlsx', sheet_name='drug1718')
drug_2005_2016.head(3)

,region,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Российская Федерация,17.2,19.1,19.1,18.7,17.8,17.6,15.4,13.9,12.6,14.5,14.1,11.1
1,Центральный федеральный округ,9.5,11.8,11.8,13.1,13.9,14.9,12.3,12.9,13.9,16.9,14.9,10.7
2,Белгородская область,4.5,9.5,9.5,4.2,3.9,4.8,4.0,2.3,2.4,1.9,2.0,1.6


После всех преобразований и обьединений наша таблица выглядит так:

In [147]:
df_drug = pd.read_csv('data/total/14_df_drug.csv')
df_drug.head(3)

,Регион,Drug
0,Белгородская область,6.49
1,Брянская область,12.36
2,Владимирская область,10.19


**Вывод: За годы заболеваемость наркоманией снизилась почти в 2 раза. В 2005 ярко выраженного лидера по заболеваниям не было, а вот в 2018 появился - это Еврейская автономная республика(25). Меньше всего на Чукотке(5). Среднее значение 12.**

### `15. Рождённые в этом месяце, по регионам, без учёта мертворождённых.`

In [148]:
newborn = pd.read_csv('data/newborn_2006_2022_monthly.csv',sep=';', decimal=',')
display(newborn.head(3))

,Region,январь 2006 г.,февраль 2006 г.,март 2006 г.,апрель 2006 г.,май 2006 г.,июнь 2006 г.,июль 2006 г.,август 2006 г.,сентябрь 2006 г.,...,сентябрь 2021 г.,октябрь 2021 г.,ноябрь 2021 г.,декабрь 2021 г.,январь 2022 г.,февраль 2022 г.,март 2022 г.,апрель 2022 г.,май 2022 г.,Unnamed: 198
0,Российская Федерация,109422.00,114068.99,129177.99,113391.99,125099.99,124238.0,125926.99,140227.99,123949.99,...,121645.99,120080.99,114976.99,117945.99,103782.0,101082.0,115512.0,99696.99,103159.0,NaN
1,Центральный федеральный округ,24998.99,25539.99,29558.99,25781.00,29137.00,28837.0,28997.99,31590.99,27755.99,...,29727.00,29532.99,29093.00,29477.00,25827.0,25678.0,28086.0,24979.99,26304.0,NaN
2,Белгородская область,1040.00,1094.00,1190.00,1044.00,1228.00,1249.0,1224.00,1389.00,1157.00,...,1084.00,1053.00,990.00,1028.00,861.0,941.0,956.0,801.00,869.0,NaN


После всех преобразований наша таблица выглядит так:

In [149]:
df_newborn = pd.read_csv('data/total/15_df_newborn.csv')
df_newborn.head(3)

,index,Регион,Newborn
0,0,Белгородская область,14792.82
1,1,Брянская область,12476.00
2,2,Владимирская область,13653.14


**Вывод: Средняя рождаемость почти не изменилась, а вот по отдельным регионам изменилась, например в Москве рождаемость выросла на 30%. В итоге самое большое значение у Москвы (95206), что в 5 раз больше среднего количества, самое маленькое значение на Чукотке. среднее значение 17741**

### `16. Сведения о преступлениях, совершённых отдельными категориями лиц за 2016–2022 гг., по месяцам, регионам, категориям лиц, категориям преступлений:`

Данные за каждый месяц агрегированы с предыдущими месяцами этого года (то есть январь 2022, далее — январь + февраль 2022 и так далее). Чтобы не смотреть все разбивки по месяцам, выгрузим последний суммарный месяц декабрь.

Вся информация в сумме приводится на первом листе каждого файла («строка 1» в терминологии документов). На остальных строках в зависимости от года приводится разбивка по категориям преступлений (особо тяжкие, тяжкие, средней тяжести, небольшой тяжести), а далее, если есть дополнительные листы (с 2021 года) — по главам и статьям Уголовного кодекса. Для более ранних годов разбивки по преступлениям нет — только по категориям преступлений.

In [150]:
crimes_2016 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122016.xls', sheet_name='Строка 1', skiprows=9)
crimes_2017 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122017.xls', sheet_name='Строка 1', skiprows=9)
crimes_2018 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122018.xls', sheet_name='Строка 1', skiprows=9)
crimes_2019 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122019.xls', sheet_name='Строка 1', skiprows=9)
crimes_2020 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122020.xls', sheet_name='Строка 1', skiprows=9)
crimes_2021 = pd.read_excel('data/crimes/4-EGS_Razdel_5_122021.xls', sheet_name='Строка 1', skiprows=9)
display(crimes_2016.head(3))

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,Unnamed: 0,Unnamed: 1,несовершеннолетними или при их соучастии\n,ранее совершавшими преступления\n,в том числе,группой лиц\n,группой лиц по предварительному сговору\n,организованной группой\n,преступным сообществом (преступной организацией)\n,в состоянии опьянения,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,ранее судимыми,NaN,NaN,NaN,NaN,алкогольного\n,наркотического\n,токсического\n
1,NaN,NaN,1.0,2.0,3,4.0,5.0,6.0,7.0,8,9,10
2,NaN,Российская Федерация,53736.0,674935.0,359171,6569.0,85216.0,11186.0,1395.0,440154,28237,200


После достаточно большой обработки данных и преобразований, я сделала две таблицы по суммарным преступлениям, а также по преступлениям, совершенными в состоянии алгокольного, наркотическгого или таксических опьянений, Дополнять недостающие года тут тоже не было смысла, тк это сильно искажало данные.

In [151]:
df_crimes_total = pd.read_csv('data/total/16_df_crimes_total.csv')
df_crimes_total.head(3)

,Регион,Сrimes_total
0,Белгородская область,9673.41
1,Брянская область,10100.14
2,Владимирская область,10536.09


In [152]:
df_crimes_drug = pd.read_csv('data/total/16_df_crimes_drug.csv')
df_crimes_drug.head(3)

,Регион,Сrimes_drug
0,Белгородская область,3587.91
1,Брянская область,3816.09
2,Владимирская область,3938.05


**Выводы: Среднее совершение преступление с 2016 года снизилось на 21%. Лидером по совершению преступлений являютя Челябинская(18800), Cвердловская области и Московская область. Меньше всего в Ненецком автономном округе (7736). В среднем этот  показатель 11141.
Следует заметить, что треть  всех преступлений совершаются в состоянии алкогольного, наркотического и токсических опъянений. Тут лидерами  являются Республика Башкортастан(6868), Свердловская и Челябинская области. Меньше всего в Чеченской республике (2982). В среднем этот показатель 4085.**

# <CENTER> ИТОГИ ПРЕДОБРАБОТКИ ДАННЫХ:

Итак, я обработала 16 таблиц, приведя всех к единому формату и количеству регионов, взяв среднее значение с 2000 по 2021 годы. Также создала новые таблицы. 

По результатам каждой обработки были написаны соответствующие выводы.

Всего у нас получилось 27 признаков, давайте обьединим все данные в одну таблицу:

In [153]:
df_01 = pd.read_csv('data/total/01_df_child_mortality_rural.csv')
df_02 = pd.read_csv('data/total/02_df_child_mortality_urban.csv')
df_03 = pd.read_csv('data/total/03_df_disabled_18_30.csv')
df_04 = pd.read_csv('data/total/03_df_disabled_31_60.csv')
df_05 = pd.read_csv('data/total/03_df_disabled_61_.csv')
df_06 = pd.read_csv('data/total/03_df_disabled_total.csv')
df_07 = pd.read_csv('data/total/04_df_morbidity_onko.csv')
df_08 = pd.read_csv('data/total/04_df_morbidity_psiho.csv')
df_09 = pd.read_csv('data/total/04_df_morbidity_total.csv')
df_10 = pd.read_csv('data/total/05_df_poverty.csv')
df_11 = pd.read_csv('data/total/06_df_welfare.csv')
df_12 = pd.read_csv('data/total/07_df_2cash.csv')
df_13 = pd.read_csv('data/total/08_df_socdem_deti_2.csv')
df_14 = pd.read_csv('data/total/08_df_socdem_pens_2.csv')
df_15 = pd.read_csv('data/total/08_df_socdem_trud_2.csv')
df_16 = pd.read_csv('data/total/09_df_housing_live.csv')
df_17 = pd.read_csv('data/total/10_df_population.csv')
df_18 = pd.read_csv('data/total/11_df_grp.csv')
df_19 = pd.read_csv('data/total/12_df_reg_prod_manufactor.csv')
df_20 = pd.read_csv('data/total/12_df_reg_prod_mining.csv')
df_21 = pd.read_csv('data/total/12_df_reg_prod_total.csv')
df_22 = pd.read_csv('data/total/13_df_retail.csv')
df_23 = pd.read_csv('data/total/14_df_alco.csv')
df_24 = pd.read_csv('data/total/14_df_drug.csv')
df_25 = pd.read_csv('data/total/15_df_newborn.csv')
df_26 = pd.read_csv('data/total/16_df_crimes_drug.csv')
df_27 = pd.read_csv('data/total/16_df_crimes_total.csv')

In [154]:
df = df_01.merge(df_02,on='Регион',how='inner')
df = df.merge(df_03,on='Регион',how='inner')
df = df.merge(df_04,on='Регион',how='inner')
df = df.merge(df_05,on='Регион',how='inner')
df = df.merge(df_06,on='Регион',how='inner')
df = df.merge(df_07,on='Регион',how='inner')
df = df.merge(df_08,on='Регион',how='inner')
df = df.merge(df_09,on='Регион',how='inner')
df = df.merge(df_10,on='Регион',how='inner')
df = df.merge(df_11,on='Регион',how='inner')
df = df.merge(df_12,on='Регион',how='inner')
df = df.merge(df_13,on='Регион',how='inner')
df = df.merge(df_14,on='Регион',how='inner')
df = df.merge(df_15,on='Регион',how='inner')
df = df.merge(df_16,on='Регион',how='inner')
df = df.merge(df_17,on='Регион',how='inner')
df = df.merge(df_18,on='Регион',how='inner')
df = df.merge(df_19,on='Регион',how='inner')
df = df.merge(df_20,on='Регион',how='inner')
df = df.merge(df_21,on='Регион',how='inner')
df = df.merge(df_22,on='Регион',how='inner')
df = df.merge(df_23,on='Регион',how='inner')
df = df.merge(df_24,on='Регион',how='inner')
df = df.merge(df_25,on='Регион',how='inner')
df = df.merge(df_26,on='Регион',how='inner')
df = df.merge(df_27,on='Регион',how='inner')
df.head()

,Регион,Child_mortality_rural,Child_mortality_urban,Disabled_18_30,Disabled_31_60,Disabled_61_,Disabled_total,Morbidity_onko,Morbidity_psiho,Morbidity_total,...,Reg_prod_manufactor,Reg_prod_mining,Reg_prod_total,Retail,Alco,Drug,index,Newborn,Сrimes_drug,Сrimes_total
0,Белгородская область,39.59,63.64,52985.91,444265.82,934390.86,1464366.05,2589.46,1566.49,41564.62,...,5.212134e+08,1.152325e+08,1.115343e+09,114915.23,91.07,6.49,0,14792.82,3587.91,9673.41
1,Брянская область,42.09,70.18,48351.36,384930.09,733906.64,1199911.55,2595.53,1564.57,43703.95,...,1.624191e+08,2.869199e+05,3.097550e+08,109399.95,145.76,12.36,1,12476.00,3816.09,10100.14
2,Владимирская область,27.95,84.77,48950.32,386048.59,780816.45,1248538.82,2588.23,1613.43,50096.00,...,3.448643e+08,3.491436e+06,6.444769e+08,87961.45,117.60,10.19,2,13653.14,3938.05,10536.09
3,Воронежская область,44.27,114.95,57292.09,478965.55,928332.91,1497314.00,2617.92,1705.19,29806.02,...,3.488616e+08,5.937693e+06,7.443726e+08,123738.77,122.90,15.02,3,19701.54,4231.77,11535.32
4,Ивановская область,15.36,66.41,45812.27,350638.14,693232.23,1122406.09,2593.44,1708.60,48225.12,...,1.141104e+08,8.266384e+05,2.403929e+08,88192.55,136.98,10.30,4,10781.22,3615.36,9632.55


In [155]:
#Удалим лишний столбец с индексом:
df = df.drop(columns=['index'])
df.shape

(85, 28)

In [156]:
#Сохраним данную таблицу для дальнейшей работы:
df.reset_index(inplace=True)
df.to_csv('data/total/df_total_2.csv', index=False)
df.head(3)

,index,Регион,Child_mortality_rural,Child_mortality_urban,Disabled_18_30,Disabled_31_60,Disabled_61_,Disabled_total,Morbidity_onko,Morbidity_psiho,...,GRP,Reg_prod_manufactor,Reg_prod_mining,Reg_prod_total,Retail,Alco,Drug,Newborn,Сrimes_drug,Сrimes_total
0,0,Белгородская область,39.59,63.64,52985.91,444265.82,934390.86,1464366.05,2589.46,1566.49,...,296532.88,5.212134e+08,1.152325e+08,1.115343e+09,114915.23,91.07,6.49,14792.82,3587.91,9673.41
1,1,Брянская область,42.09,70.18,48351.36,384930.09,733906.64,1199911.55,2595.53,1564.57,...,159780.83,1.624191e+08,2.869199e+05,3.097550e+08,109399.95,145.76,12.36,12476.00,3816.09,10100.14
2,2,Владимирская область,27.95,84.77,48950.32,386048.59,780816.45,1248538.82,2588.23,1613.43,...,188789.51,3.448643e+08,3.491436e+06,6.444769e+08,87961.45,117.60,10.19,13653.14,3938.05,10536.09
